In [75]:
import pandas as pd
import argparse
from collections import defaultdict
import datetime
from pprint import pprint
from google.cloud import datastore
import google.cloud.exceptions
import copy

from multiprocessing import Pool

In [2]:
client = datastore.Client()

### Retrieve by key

In [41]:
res = client.query(kind="node")
first_key = client.key("node","0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899")
# first_key = client.key("node","test")
res.key_filter(first_key,"=")
data = list(res.fetch())

In [65]:
def get_data(cid):
    key = client.key("node",cid)
    task = client.get(key)
    
    global results, cnt
    
    cnt += 1
    
    for e in task["edges"]:
        if e not in results:
            results.append(e)
    

In [ ]:
%%time
results = data["edges"]
cnt = 0

for e in data["edges"]:
#     print(e)
    get_data(e)

In [10]:
len(data[0]["edges"])

238

In [99]:
def get_node(cid):
    return client.get(client.key("node",cid))["edges"]

In [105]:
def next_degree(edges):
    
    results = copy.copy(edges)
    
    with Pool(16) as p:
        results = p.map(get_node,edges)

#     for e in edges:
#         res = client.get(client.key("node",e))["edges"]
#         results.append(res)

    dedup = []
    
    for r in results:
        if type(r) == str:
            if r not in dedup:
                dedup.append(r)
        else:
            for i in r:
                if i not in dedup:
                    dedup.append(i)
                    
    return dedup

In [113]:
%%time
d1 = data[0]["edges"]
d2 = next_degree(d1)
print("D2 edges {}".format(len(d2)))
d3_lookup = list(set(d2).difference(d1))
d3 = next_degree(d3_lookup)
# d3 = list(set(d3).union(d2))
print("D3 edges {}".format(len(d3)))
d4_lookup = list(set(d3).difference(d2))
d4 = next_degree(d4_lookup)
print("D4 edges {}".format(len(d4)))

D2 edges 24
D3 edges 7319
D4 edges 2830
CPU times: user 506 ms, sys: 375 ms, total: 881 ms
Wall time: 6.34 s


In [ ]:
# 262
# 7333
# 9926
# CPU times: user 14 s, sys: 1.81 s, total: 15.8 s
# Wall time: 1min 25s

In [111]:
%%time
d_test = next_degree(d4_lookup)

CPU times: user 288 ms, sys: 252 ms, total: 540 ms
Wall time: 5.32 s


In [112]:
%%time

len(d_test)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.44 µs


2824

10156

In [85]:
%%time

def test(x):
    return len(x)

test_arr = []

for e in d4:
    test_arr.append(test(e))

# print(test_arr)
    
# with Pool(8) as p:
#     test_arr.append(test(e))

CPU times: user 3.17 ms, sys: 0 ns, total: 3.17 ms
Wall time: 3.15 ms


In [98]:
%%time

def test(x):
    return len(x)

test_arr = []

with Pool(8) as p:
    test_arr = p.map(next_degree,d4_lookup)
    
print(test_arr)

TypeError: 'NoneType' object is not subscriptable

In [97]:
print(test_arr)

[42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42,

In [4]:
def gquery(id):
    res = client.query(kind="node")
    first_key = client.key("node",id)
    # first_key = client.key("node","test")
    res.key_filter(first_key,"=")
    data = list(res.fetch())
#     print(data[0])

In [5]:
%%time
for e in data[0]["edges"]:
    gquery(e)

CPU times: user 528 ms, sys: 70.2 ms, total: 598 ms
Wall time: 21.5 s


In [23]:
def ds_get(key):

    return client.get(client.key("node",key))

In [34]:
def get_edges(array, edges):
    
    global counter
    
    print(len(edges))
    
    for e in edges:
    #     print(e)
    #     print("---------------------------------test---------------------------------")
        
        counter += 1
        sacc = ds_get(e)
    #     print(sacc)
        for i in sacc["edges"]:
            array.append(i)
#             if i not in array:
#                 array.append(i)
                
    return array
    

In [35]:
%%time
gedges = []
counter = 0

pacc = ds_get("0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899")
# print(len(pacc["edges"]))
# print(pacc)
#     print(pacc["edges"])
# 1st degree
gedges = get_edges(gedges,pacc["edges"])
print("# of Edges {}, # of Counter {}".format(len(gedges),counter))
# counter = 0
# # 2nd degree
# gedges = get_edges(gedges,gedges)
# print("# of Edges {}, # of Counter {}".format(len(gedges),counter))
# counter = 0
# # 3rd degree
# gedges = get_edges(gedges,gedges)
# print("# of Edges {}, # of Counter {}".format(len(gedges),counter))
# counter = 0

    
print(gedges)

238
# of Edges 271, # of Counter 238
['0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0x02d048b763e7cc9b790fa504ee83b4f225472e90', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0xf4e364114aaf619e94757fbf4168084523365b7b', '0xcd83fd091a48ef72f4f4f170b68f68f1a098a9b0', '0x3974d816a969908a82428384c728fbd8faffbfe1', '0x2b5634c42055806a59e9107ed44d43c426e58258', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0x05ee546c1a62f90d7acbffd6d846c9c54c7cf94c', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d5899', '0xf3e36ad56aa85abdacc18c02d19509ae4f7d589

In [12]:
set(edges)

24

In [ ]:
<Entity('node', '0x0000000048246c5f37df029bef0cefd9ee36f301') 
    {'mobile_pager_number': 4882, 'count_of_non_rbwm_csem_connections': 0, 
     'count_of_sending': 7, 'count_of_receiving': 0, 
     'edges': ['0xe87729bcf07cd2be7333a393b94c0959e30d9f96', '0xe87729bcf07cd2be7333a393b94c0959e30d9f96', 
               '0xe87729bcf07cd2be7333a393b94c0959e30d9f96', '0xe87729bcf07cd2be7333a393b94c0959e30d9f96', 
               '0xe87729bcf07cd2be7333a393b94c0959e30d9f96', '0xe87729bcf07cd2be7333a393b94c0959e30d9f96', 
               '0xe87729bcf07cd2be7333a393b94c0959e30d9f96', '0xe87729bcf07cd2be7333a393b94c0959e30d9f96'], 
     'email': '8526@gmail.com', 
     'ip_address_array': '["241.92.245.185","188.22.55.210","58.215.140.56","244.217.99.101","39.207.152.141"]', 'amount_of_receiving': 0.0, 'count_of_rbwm_caut_connections': 0, 'count_of_rbwm_csem_connections': 1, 'join_date': '3/27/2008 0:00', 'amount_of_sending': 0.100490075, 'last_datetime': '1/15/2019 9:00', 'count_of_non_rbwm_caut_connections': 0, 'first_datetime': '1/15/2019 9:00', 'home_tel_number': 2092, 'count_of_total_connections': 1}>

In [36]:
len(gedges)

271

In [137]:
gedges[89]

['0xf7b09b87781db996a0eabfe23c8ca7682c26f61d',
 '0xf740651b287938dae1431d9f0c99d1c45ff2c5cc',
 '0xf70e873087b8aab2d00914482d5368a6412a4b9c',
 '0xf7000762664afb5e615a93fd1d791e3dc4c7b43c',
 '0xf690ffb6eea99d9885070bbca4b22c009dd2de33',
 '0xf3e84b17d68c371519ba41676520d1c45190c8ed',
 '0xf3b737219e17a962cc1eb5487bb2692b0c49da92',
 '0xeff73f6b8a247922041ebb92592c258d6553dd53',
 '0xef0008e5a8daf0613aa5d9a8e0de83eba8387bfb',
 '0xea149090a332be6f0432270ed977a5e9edacaca6',
 '0xe985e55a345ad6524b3d182f189ddeb067b61b36',
 '0xe75cd7a178459006f79fdc188ef294f3e7254d49',
 '0xd9f15faae5dd3a361af2d326ed747cf36a8637bb',
 '0xd34093334303555aa92cf4768b6c294e6d506ace',
 '0xd1bb79996c59d9aa6b80cf5fcb8955ec4228966d',
 '0xd188438a702fe00ab14e4980a7e4aabb29e67772',
 '0xcdd2333c2455d44c12d11c59c4dc0035715a9d88',
 '0xccd4205280ee65f58199347c99ba6b6bace8d17c',
 '0xca7c709884149be65bc9a95faae59f89bde24255',
 '0xc9a9e88c37ec87807ddef79c6b5f972f3068377b',
 '0xc9626dffe54c8ba4c21518ff20c4b6166ec23ab4',
 '0xc8d2fe246

In [113]:
res = client.query(kind="edges")
# first_key = client.key("node","0x00192fb10df37c9fb26829eb2cc623cd1bf599e8")
# # first_key = client.key("node","test")
# res.key_filter(first_key,"=")
data = list(res.fetch())
data

[]

### Test the datastore insert

Override entire document

In [79]:
key = client.key("node","test")
task = datastore.Entity(key)
task.update(
    {
        "email":"test@gmail.com",
        "home_tel_number":"12321321",
        "mobile_pager_number":"12321321"
    }
)
client.put(task)

Update specific property of the entity

In [92]:
with client.transaction():
    key = client.key("node","test")
    task = client.get(key)
    
#     print(task)
    if "edges" in task:
        edges = task["edges"]
        edges.insert(0,"edge_test123")
    else:
        edges = ["testing321"]
    
    task["edges"] = edges
    
    client.put(task)


<Entity('node', 'test') {'home_tel_number': '12321321', 'email': 'test@gmail.com', 'mobile_pager_number': '12321321', 'edges': ['testing321']}>


### Edge insert

In [47]:
edges = pd.read_csv("/tmp/links.csv")

In [48]:
edges.head()

,sender,receiver,sender_cds_caut,receiver_cds_caut,sender_cds_csem,receiver_cds_csem,sender_csem_exit,receiver_csem_exit,first_datetime,last_datetime,count_of_sending,count_of_receiving,amount_of_sending,amount_of_receiving
0,0x0000000048246c5f37df029bef0cefd9ee36f301,0xe87729bcf07cd2be7333a393b94c0959e30d9f96,0,0,0,1,NaN,NaN,5/15/2018 9:00,1/15/2019 9:00,7,0,0.100490,0.0
1,0x00004225550a3942d1877a3a43df923eb5279a5b,0xb3aaaae47070264f3595c5032ee94b620a583a39,0,0,1,0,NaN,NaN,5/15/2018 9:00,1/15/2019 9:00,7,0,0.347569,0.0
2,0x00008e23e286645c9c1e13ec3b02e18d0ddd1d7d,0xe1134bfd57131ce3ecfe69300b39728fd5c51573,0,0,0,0,NaN,NaN,5/15/2018 9:00,1/15/2019 9:00,7,0,2.297845,0.0
3,0x0001f2fe9bc9e12b975d575f67321121e2579b6d,Yobit,0,0,0,0,NaN,NaN,5/15/2018 9:00,1/15/2019 9:00,7,0,0.342718,0.0
4,0x00029638d51bd8ef7da7f43911aa9aaafe28dc7e,0xdc25e92d0f361f54052a1a4ad934b9184f13b255,0,0,1,0,NaN,NaN,5/15/2018 9:00,1/15/2019 9:00,7,0,0.106384,0.0


In [49]:
edges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219806 entries, 0 to 219805
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   sender               219806 non-null  object 
 1   receiver             219806 non-null  object 
 2   sender_cds_caut      219806 non-null  int64  
 3   receiver_cds_caut    219806 non-null  int64  
 4   sender_cds_csem      219806 non-null  int64  
 5   receiver_cds_csem    219806 non-null  int64  
 6   sender_csem_exit     0 non-null       float64
 7   receiver_csem_exit   0 non-null       float64
 8   first_datetime       219806 non-null  object 
 9   last_datetime        219806 non-null  object 
 10  count_of_sending     219806 non-null  int64  
 11  count_of_receiving   219806 non-null  int64  
 12  amount_of_sending    219806 non-null  float64
 13  amount_of_receiving  219806 non-null  float64
dtypes: float64(4), int64(6), object(4)
memory usage: 23.5+ MB


In [54]:
def gnode_edge_update(client, sender, receiver):
    
    key = client.key("node",sender)
    task = datastore.Entity(key)
    task.update(
        {
            "edges":[
                "{}".format(receiver)
            ]
        }
    )
    client.put(task)

In [55]:
def gedge_create(client, row):
    gnode_edge_update(client, row["sender"], row["receiver"])
    gnode_edge_update(client, row["receiver"], row["sender"])
    
    key = client.key("edge")
    task = datastore.Entity(key)
    task.update(
        {
            "sender":row["sender"],
            "receiver":row["receiver"],
            "sender_cds_caut":row["sender_cds_caut"],
            "receiver_cds_caut":row["receiver_cds_caut"],
            "sender_cds_csem":row["sender_cds_csem"],
            "receiver_cds_csem":row["receiver_cds_csem"],
            "sender_csem_exit":row["sender_csem_exit"],
            "receiver_csem_exit":row["receiver_csem_exit"],
            "first_datetime":row["first_datetime"],
            "last_datetime":row["last_datetime"],
            "count_of_sending":row["count_of_sending"],
            "count_of_receiving":row["count_of_receiving"],
            "amount_of_sending":row["amount_of_sending"],
            "amount_of_receiving":row["amount_of_receiving"]
        }
    )
    client.put(task)

In [56]:
for index, row in edges.iterrows():
    gedge_create(client,row)

In [57]:
edges.head(n=2)

,sender,receiver,sender_cds_caut,receiver_cds_caut,sender_cds_csem,receiver_cds_csem,sender_csem_exit,receiver_csem_exit,first_datetime,last_datetime,count_of_sending,count_of_receiving,amount_of_sending,amount_of_receiving
0,0x0000000048246c5f37df029bef0cefd9ee36f301,0xe87729bcf07cd2be7333a393b94c0959e30d9f96,0,0,0,1,NaN,NaN,5/15/2018 9:00,1/15/2019 9:00,7,0,0.100490,0.0
1,0x00004225550a3942d1877a3a43df923eb5279a5b,0xb3aaaae47070264f3595c5032ee94b620a583a39,0,0,1,0,NaN,NaN,5/15/2018 9:00,1/15/2019 9:00,7,0,0.347569,0.0
